### ДЗ #6 Классификация текста/

Для начала, хотелось бы поработать над этой задачей с собственным датасетом.. На курсе "ML в бизнесе" делал курсовую по похожей теме, и, чтобы не разрабатывать sentiment analysis на уже порядком надоевшем IMDB - заскрапил базу отзывов с kinopoinsk.ru Там правда классификация на три класса - но это может даже интереснее. Вобщем с позволения - дз буду делать на этом датасете.. 

Сама курсовая, если вдруг интересно - тут:
https://github.com/po-stas/review_analyzer

In [2]:
# Выкачиваем датасет
!mkdir datasets
!wget -O ./datasets/kinopoisk.csv https://raw.githubusercontent.com/po-stas/review_analyzer/master/kinopoisk.csv

mkdir: cannot create directory ‘datasets’: File exists
--2021-04-16 13:41:49--  https://raw.githubusercontent.com/po-stas/review_analyzer/master/kinopoisk.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7879478 (7,5M) [text/plain]
Saving to: ‘./datasets/kinopoisk.csv’

./datasets/kinopois 100%[===================>]   7,51M  2,94MB/s    in 2,6s    

2021-04-16 13:41:53 (2,94 MB/s) - ‘./datasets/kinopoisk.csv’ saved [7879478/7879478]



In [1]:
import pandas as pd
import numpy as np
from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import decomposition, ensemble
import nltk
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
import re
import textblob, string
from wordcloud import WordCloud
import matplotlib.pyplot as plt
from pymorphy2 import MorphAnalyzer
from stop_words import get_stop_words
from string import punctuation
from sklearn.model_selection import train_test_split

In [2]:
dataset = pd.read_csv('datasets/kinopoisk.csv')
dataset.head(3)

,Unnamed: 0,grade,text
0,0,1.0,Гениальный студент Ягами Лайт с помощью тетрад...
1,1,1.0,"Как известно, в СССР кино было делом государст..."
2,2,1.0,На фоне многих других фильмов Марвел первые д...


In [3]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1710 entries, 0 to 1709
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  1710 non-null   int64  
 1   grade       1710 non-null   float64
 2   text        1710 non-null   object 
dtypes: float64(1), int64(1), object(1)
memory usage: 40.2+ KB


In [4]:
# Проверим распределение целевой переменной
dataset['grade'].value_counts(normalize=True)

1.0    0.730409
0.5    0.147953
0.0    0.121637
Name: grade, dtype: float64

In [5]:
# Сильный дисбаланс классов в сторону положительных отзывов

In [6]:
# Для начала - бейзлайн какой-нибудь
# Подготовим корпус

In [7]:
lemmer = MorphAnalyzer()

In [8]:
def token_prep(token):
    return lemmer.parse(''.join([char for char in token if char.isalpha()]).lower())[0].normal_form

In [9]:
prep_data = dataset.copy()

In [10]:
prep_data['text'] = prep_data['text'].apply(lambda x: ' '.join([token_prep(token) for token in x.split()]))

In [11]:
prep_data.text.head(3)

0    гениальный студент яга лайта с помощь тетрадь ...
1    как известно в ссср кино быть дело государстве...
2    на фон многий другой фильм марвести первый два...
Name: text, dtype: object

In [12]:
X_train, X_test, y_train, y_test = train_test_split(prep_data.text, prep_data.grade, 
                                                    test_size=0.2, random_state=42)

In [13]:
X_train.head()

838     обожать этот фильм у француз в начало х сохран...
570     вот это уже большой похоже на южный парк есть ...
873     второй картина гай ричать сохранить стилистиче...
1557    крайне эффектный по свой атмосфера мрачный пуг...
426     отличный фильм первый раз посмотреть он перед ...
Name: text, dtype: object

In [14]:
# Векторизация
tfvectorizer = TfidfVectorizer(ngram_range=(1, 4), analyzer='char')

In [15]:
X_train_tf = tfvectorizer.fit_transform(X_train)
X_test_tf = tfvectorizer.transform(X_test)
X_train_tf.shape

(1368, 74037)

In [16]:
le = LabelEncoder()

In [17]:
y_train = le.fit_transform(y_train)

In [18]:
y_test = le.transform(y_test)

In [19]:
%%time
lr = LogisticRegression(random_state=42, max_iter=100)
lr.fit(X_train_tf, y_train)

CPU times: user 28.2 s, sys: 20.6 s, total: 48.7 s
Wall time: 3.54 s


LogisticRegression(random_state=42)

In [20]:
pred_tf = lr.predict(X_test_tf)

In [21]:
accuracy_score(y_test, pred_tf)

0.7397660818713451

In [22]:
# Вообще, наверное при такой несбалансированности классов - тут надо измерять ROC_AUC

In [23]:
roc_auc_score(y_test, lr.predict_proba(X_test_tf), multi_class='ovr')

0.7395840828013317

In [24]:
# Понятно.. Как-то примерно так и было в курсовой по Bussiness ML
# Попробуем улучшить результат
# Для начала - попробуем поиспользовать эмбеддинги

In [25]:
from gensim.models import Word2Vec

/home/postas/anaconda3/lib/python3.8/site-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [158]:
prep_data_emb = dataset.copy()

In [159]:
prep_data_emb['text'] = prep_data_emb['text'].apply(lambda x: [token_prep(token) for token in x.split()])

In [273]:
prep_data_emb = prep_data_emb[prep_data_emb['text'].map(len) > 0]

In [276]:
# Бьём на трейн-тест
X_train_emb, X_test_emb, y_train_emb, y_test_emb = train_test_split(prep_data_emb.text, prep_data_emb.grade, 
                                                    test_size=0.2, random_state=42, shuffle=True)

In [277]:
# Учим word2vec
w2v = Word2Vec(X_train_emb, vector_size=1000, workers=16, window=5)

In [278]:
w2v.wv['гениальный']

array([ 1.28432840e-01,  1.47766834e-02,  4.15462814e-02,  7.27694556e-02,
       -2.89174076e-03, -4.86377254e-02,  2.88003385e-02,  6.77371547e-02,
       -8.74591153e-03, -1.31516310e-03,  6.34485781e-02,  6.57770736e-03,
       -3.76283424e-03,  1.30032999e-02,  6.73472434e-02, -3.18785459e-02,
        6.59571867e-03, -3.99084277e-02,  1.46304769e-02, -1.27435520e-01,
        7.40185082e-02, -1.16130106e-01,  3.16223912e-02, -1.89955123e-02,
        7.67104924e-02, -6.64684027e-02,  4.77185026e-02, -3.20074707e-02,
       -1.85293511e-01,  4.65369560e-02,  7.22886696e-02, -6.63369372e-02,
       -5.54299206e-02, -1.16584711e-02,  1.21556945e-01, -4.84180152e-02,
        8.24535489e-02, -2.69778967e-02, -4.85867858e-02, -1.14521690e-01,
       -5.64203598e-02,  3.73579599e-02, -1.23828031e-01,  1.43307760e-01,
       -4.85709161e-02, -1.53342541e-02, -9.50107947e-02,  1.66504428e-01,
       -7.48955905e-02,  5.44472001e-02,  2.55961884e-02,  1.96285732e-02,
       -1.97715852e-02, -

In [279]:
# Что-то есть..
# Прогоним это в LogReg. Для сравнения

In [280]:
X_train_emb = X_train_emb.apply(lambda x: np.mean([w2v.wv[emb] for emb in x if emb in w2v.wv], axis=0))

In [282]:
X_test_emb = X_test_emb.apply(lambda x: np.mean([w2v.wv[emb] for emb in x if emb in w2v.wv], axis=0))

In [283]:
# Лейблы
y_train_emb = le.fit_transform(y_train_emb)
y_test_emb = le.fit_transform(y_test_emb)

In [291]:
%%time
lr = LogisticRegression(random_state=42, max_iter=1000)
lr.fit(np.stack(X_train_emb, axis=0), y_train_emb)

CPU times: user 2.18 s, sys: 1.62 s, total: 3.79 s
Wall time: 252 ms


LogisticRegression(max_iter=1000, random_state=42)

In [292]:
roc_auc_score(y_test_emb, lr.predict_proba(np.stack(X_test_emb, axis=0)), multi_class='ovr')

0.6580411732594271

In [294]:
# И снова, как-то тухло ))) Ембеддинги не рулят почему-то в моих экспериментах никак
# Попробую еще размер вектора сделать длиннее

In [295]:
# Бьём на трейн-тест снова
X_train_emb, X_test_emb, y_train_emb, y_test_emb = train_test_split(prep_data_emb.text, prep_data_emb.grade, 
                                                    test_size=0.2, random_state=42, shuffle=True)

In [296]:
# Учим word2vec
w2v = Word2Vec(X_train_emb, vector_size=5000, workers=16, window=3)

In [297]:
X_train_emb = X_train_emb.apply(lambda x: np.mean([w2v.wv[emb] for emb in x if emb in w2v.wv], axis=0))

In [298]:
X_test_emb = X_test_emb.apply(lambda x: np.mean([w2v.wv[emb] for emb in x if emb in w2v.wv], axis=0))

In [299]:
# Лейблы
y_train_emb = le.fit_transform(y_train_emb)
y_test_emb = le.fit_transform(y_test_emb)

In [300]:
%%time
lr = LogisticRegression(random_state=42, max_iter=1000)
lr.fit(np.stack(X_train_emb, axis=0), y_train_emb)

CPU times: user 16.2 s, sys: 4.41 s, total: 20.6 s
Wall time: 1.33 s


LogisticRegression(max_iter=1000, random_state=42)

In [301]:
roc_auc_score(y_test_emb, lr.predict_proba(np.stack(X_test_emb, axis=0)), multi_class='ovr')

0.6515286176198875

In [302]:
# Хуже
# Попробуем другие параметры
X_train_emb, X_test_emb, y_train_emb, y_test_emb = train_test_split(prep_data_emb.text, prep_data_emb.grade, 
                                                    test_size=0.2, random_state=42, shuffle=True)
w2v = Word2Vec(X_train_emb, vector_size=500, workers=16, window=10)
X_train_emb = X_train_emb.apply(lambda x: np.mean([w2v.wv[emb] for emb in x if emb in w2v.wv], axis=0))
X_test_emb = X_test_emb.apply(lambda x: np.mean([w2v.wv[emb] for emb in x if emb in w2v.wv], axis=0))
y_train_emb = le.fit_transform(y_train_emb)
y_test_emb = le.fit_transform(y_test_emb)
lr = LogisticRegression(random_state=42, max_iter=1000)
lr.fit(np.stack(X_train_emb, axis=0), y_train_emb)
roc_auc_score(y_test_emb, lr.predict_proba(np.stack(X_test_emb, axis=0)), multi_class='ovr')

0.6687790795330478

In [323]:
X_train_emb, X_test_emb, y_train_emb, y_test_emb = train_test_split(prep_data_emb.text, prep_data_emb.grade, 
                                                    test_size=0.2, random_state=42, shuffle=True)
w2v = Word2Vec(X_train_emb, vector_size=600, workers=16, window=6)
X_train_emb = X_train_emb.apply(lambda x: np.mean([w2v.wv[emb] for emb in x if emb in w2v.wv], axis=0))
X_test_emb = X_test_emb.apply(lambda x: np.mean([w2v.wv[emb] for emb in x if emb in w2v.wv], axis=0))
y_train_emb = le.fit_transform(y_train_emb)
y_test_emb = le.fit_transform(y_test_emb)
lr = LogisticRegression(random_state=42, max_iter=1000)
lr.fit(np.stack(X_train_emb, axis=0), y_train_emb)
roc_auc_score(y_test_emb, lr.predict_proba(np.stack(X_test_emb, axis=0)), multi_class='ovr')

0.6682629965566473

In [324]:
# Вобщем до TFIDF ембеддингам как до луны )

In [12]:
# Попробуем простые сеточки
import tensorflow as tf
from tensorflow.keras import Model, Sequential
from tensorflow.keras.layers import Activation, Dense, Dot, Embedding, Flatten, GlobalAveragePooling1D, Reshape
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
from tensorflow.python.eager import context
from tensorflow.keras import backend as K

tf.config.threading.set_inter_op_parallelism_threads(16)
_ = tf.Variable([1])

In [25]:
model = Sequential([
    Dense(2048, activation='relu'),
    Dense(512),
    Dense(3, activation='softmax')])

In [26]:
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir="logs")

In [27]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [28]:
model.fit(
    x = pd.DataFrame.sparse.from_spmatrix(X_train_tf),
    y = y_train,
    batch_size=64,
    validation_data=(pd.DataFrame.sparse.from_spmatrix(X_test_tf), y_test),
    epochs=15,
    callbacks=[tensorboard_callback])

Epoch 1/15
22/22 [==============================] - 2s 80ms/step - loss: 0.8412 - accuracy: 0.7211 - val_loss: 0.7101 - val_accuracy: 0.7398
Epoch 2/15
22/22 [==============================] - 1s 36ms/step - loss: 0.5544 - accuracy: 0.7540 - val_loss: 0.7825 - val_accuracy: 0.7427
Epoch 3/15
22/22 [==============================] - 1s 37ms/step - loss: 0.1197 - accuracy: 0.9725 - val_loss: 0.8650 - val_accuracy: 0.7105
Epoch 4/15
22/22 [==============================] - 1s 35ms/step - loss: 0.0356 - accuracy: 0.9965 - val_loss: 1.6795 - val_accuracy: 0.7427
Epoch 5/15
22/22 [==============================] - 1s 40ms/step - loss: 0.0027 - accuracy: 1.0000 - val_loss: 1.2637 - val_accuracy: 0.7661
Epoch 6/15
22/22 [==============================] - 1s 39ms/step - loss: 9.9566e-04 - accuracy: 1.0000 - val_loss: 1.3143 - val_accuracy: 0.7661
Epoch 7/15
22/22 [==============================] - 1s 36ms/step - loss: 0.0022 - accuracy: 1.0000 - val_loss: 1.3366 - val_accuracy: 0.7632
Epoch 8/1

In [29]:
%load_ext tensorboard
%tensorboard --logdir logs

In [30]:
# Нам нужен AUROC
preds_nn = model.predict_proba(pd.DataFrame.sparse.from_spmatrix(X_test_tf))

/home/postas/anaconda3/lib/python3.8/site-packages/tensorflow/python/keras/engine/sequential.py:425: UserWarning: `model.predict_proba()` is deprecated and will be removed after 2021-01-01. Please use `model.predict()` instead.
  warnings.warn('`model.predict_proba()` is deprecated and '


In [31]:
roc_auc_score(y_test, preds_nn, multi_class='ovr')

0.7662143474856812

In [ ]:
# Вобщем хоть и чуть-чуть выше - но все-же, ничего прорывного получить не удалось..

In [13]:
# Еще есть вариант через сеточные эмбеддинги. Тоже интересно проверить.

In [32]:
prep_data_emb = dataset.copy()
prep_data_emb['text'] = prep_data_emb['text'].apply(lambda x: [token_prep(token) for token in x.split()])
prep_data_emb = prep_data_emb[prep_data_emb['text'].map(len) > 0]

In [42]:
prep_data_emb['joined'] = prep_data_emb.text.apply(lambda x: ' '.join(x))

In [203]:
# Бьём на трейн-тест
X_train_emb, X_test_emb, y_train_emb, y_test_emb = train_test_split(prep_data_emb.joined, prep_data_emb.grade, 
                                                    test_size=0.2, random_state=42, shuffle=True)

In [204]:
X_test_emb.head()

568     для свой произведение василь быков брать сюжет...
1327    с точка зрение общий впечатление от сериал  я ...
1062    в день премьера в близкий кинотеатр всё билет ...
115     на удивление картина не разочаровать и для сто...
454     человек всегда обычно ждать наступление новый ...
Name: joined, dtype: object

In [205]:
# лейблы не закодированы
le = LabelEncoder()
y_train_emb = le.fit_transform(y_train_emb)
y_test_emb = le.transform(y_test_emb)

In [206]:
# Измерить количество слов в рецензии
prep_data_emb['word_count'] = prep_data_emb.text.apply(lambda x: len(x))

In [207]:
prep_data_emb.word_count.min()

48

In [208]:
prep_data_emb.word_count.max()

1025

In [209]:
prep_data_emb.word_count.mean()

375.03861907548276

In [210]:
prep_data_emb.word_count.median()

316.0

In [211]:
prep_data_emb.word_count.quantile(0.8)

520.0

In [212]:
# Ну, пусть будет длина сиквенса 500 слов
# Еще, не вредно будет понять размер словаря..

In [213]:
vocab = set()
for row in X_train_emb:
    for word in row.split():
        if word not in vocab:
            vocab.add(word)

In [214]:
len(vocab)

30215

In [259]:
# Вобщем....
seq_len = 500
vocab_size = len(vocab)

In [260]:
vectorize_layer = TextVectorization(
    standardize=None,  # Уже весь препроцессинг сделан
    max_tokens=vocab_size,
    output_mode='int',
    output_sequence_length=seq_len)

In [261]:
vectorize_layer.adapt(X_train_emb.to_list())

In [262]:
embedding_dim=500

model = Sequential([
  vectorize_layer,
  Embedding(vocab_size, embedding_dim, name="embedding"),
  GlobalAveragePooling1D(),
  Dense(32, activation='relu'),
  Dense(3, activation='softmax')
])

In [263]:
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir="logs")

In [264]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [265]:
model.fit(
    x = X_train_emb,
    y = y_train_emb,
    batch_size=64,
    validation_data=(X_test_emb, y_test_emb),
    epochs=10,
    callbacks=[tensorboard_callback])

Epoch 1/10
22/22 [==============================] - 3s 126ms/step - loss: 1.0355 - accuracy: 0.6695 - val_loss: 0.8069 - val_accuracy: 0.7368
Epoch 2/10
22/22 [==============================] - 3s 117ms/step - loss: 0.7845 - accuracy: 0.7335 - val_loss: 0.7585 - val_accuracy: 0.7368
Epoch 3/10
22/22 [==============================] - 3s 114ms/step - loss: 0.7802 - accuracy: 0.7169 - val_loss: 0.7444 - val_accuracy: 0.7368
Epoch 4/10
22/22 [==============================] - 3s 114ms/step - loss: 0.7232 - accuracy: 0.7355 - val_loss: 0.7361 - val_accuracy: 0.7368
Epoch 5/10
22/22 [==============================] - 3s 116ms/step - loss: 0.7241 - accuracy: 0.7191 - val_loss: 0.7247 - val_accuracy: 0.7368
Epoch 6/10
22/22 [==============================] - 3s 115ms/step - loss: 0.6720 - accuracy: 0.7304 - val_loss: 0.7098 - val_accuracy: 0.7368
Epoch 7/10
22/22 [==============================] - 3s 115ms/step - loss: 0.6384 - accuracy: 0.7153 - val_loss: 0.6872 - val_accuracy: 0.7368
Epoch 

In [266]:
preds_nn = model.predict_proba(X_test_emb)
roc_auc_score(y_test_emb, preds_nn, multi_class='ovr')

/home/postas/anaconda3/lib/python3.8/site-packages/tensorflow/python/keras/engine/sequential.py:425: UserWarning: `model.predict_proba()` is deprecated and will be removed after 2021-01-01. Please use `model.predict()` instead.
  warnings.warn('`model.predict_proba()` is deprecated and '


0.7771483161165701

In [ ]:
# Ну... еще процентик отыграть можно )

In [267]:
%load_ext tensorboard
%tensorboard --logdir logs

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


Reusing TensorBoard on port 6006 (pid 10252), started 1:04:41 ago. (Use '!kill 10252' to kill it.)

### Выводы..
Вобщем задача не проста ) Возможно у нас мало данных в датасете, чтобы хорошо натренировать классификацию, возможно стоит думать в сторону разработки дополнительных фичей, более навороченных моделей, поскольку дефолтные методы, что в классических ML подходах, что в нейросетях дают сравнимые результаты. 

Эмбеддинги Word2Vec что-то совсем не показали...